In [1]:
#general imports 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
import random
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

# imports for FMCLP algorithm
from fmclp import cuae, fmclp, synthetic_dataset, write_res,general_results_write, benefit, run_experiment

In [2]:
# preprocessing one of compas datasets

d = pd.read_csv('compas-scores-raw.csv')
                
del d['Person_ID']
del d['AssessmentID']
del d['Case_ID']
del d['LastName']
del d['MiddleName']
del d['FirstName']
del d['RawScore']
del d['DecileScore']
del d['IsCompleted']
del d['IsDeleted']
del d['AssessmentReason']
del d['RecSupervisionLevelText']
del d['DisplayText']

del d['Screening_Date']
del d['DateOfBirth']

def race(x):
    if x == 'African-American':
        return 1
    else:
        return 0
d['attr']= d['Ethnic_Code_Text'].apply(race)
del d['Ethnic_Code_Text']

def targeter(x):
    if x == 'Low':
        return 0
    elif x =='Medium':
        return 1
    else:
        return 2
d['target'] = d['ScoreText'].apply(targeter)
del d['ScoreText']
d = pd.get_dummies(d, drop_first = True)

In [3]:
d

,ScaleSet_ID,RecSupervisionLevel,Scale_ID,attr,target,Agency_Text_DRRD,Agency_Text_PRETRIAL,Agency_Text_Probation,Sex_Code_Text_Male,ScaleSet_Risk and Prescreen,...,CustodyStatus_Prison Inmate,CustodyStatus_Probation,CustodyStatus_Residential Program,MaritalStatus_Married,MaritalStatus_Separated,MaritalStatus_Significant Other,MaritalStatus_Single,MaritalStatus_Unknown,MaritalStatus_Widowed,AssessmentType_New
0,22,1,7,0,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,1
1,22,1,8,0,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,1
2,22,1,18,0,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,1
3,22,1,7,0,0,0,1,0,1,1,...,0,0,0,1,0,0,0,0,0,1
4,22,1,8,0,0,0,1,0,1,1,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60838,22,3,8,1,2,0,0,1,1,1,...,0,0,0,0,0,0,1,0,0,1
60839,22,3,18,1,2,0,0,1,1,1,...,0,0,0,0,0,0,1,0,0,1
60840,22,1,7,0,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,1
60841,22,1,8,0,1,0,1,0,1,1,...,0,0,0,0,0,0,1,0,0,1


In [4]:
#fit LGBM classifier
y = d.drop('target',axis=1)
x = d['target']
    
y_train,y_test,x_train,x_test = train_test_split(y,x,test_size = 0.3)

estimator = LGBMClassifier()
estimator.fit(y_train,x_train)
estimator_pred= estimator.predict(y_test)
accuracy_score(estimator_pred,x_test)

0.8325754670465129

In [7]:
%%time
experiment = run_experiment(dataset = d,
                   number_experiments = 100,
                   multiplier = 35,
                   number_iterations = 20,
                   interior_classifier = 'knn',
                   folder = 'compas',
                   dataset_name='compas',
                   initial_classifier = LGBMClassifier())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
CPU times: user 9h 55min 31s, sys: 45min 13s, total: 10h 40min 44s
Wall time: 2h 27min 36s


In [6]:
%%time

results = []
number_experiments = 100
for i in range(number_experiments):
    main_state = random.choice(range(1000))
    multiplier = 35
    interior_classifier = 'knn'
    res = fmclp(dataset = d, 
               estimator = LGBMClassifier(), 
               number_iterations = 20, 
               prefit = False, 
               interior_classifier = interior_classifier,
               verbose = False, 
               multiplier=multiplier, 
               random_state = main_state)
    results.append(res)
    name = f"compas/compas_№'{i+1}"
    write_res(res=res,
              name=name,
              main_state = main_state,
              multiplier= multiplier, 
              interior_classifier = interior_classifier)
    print(i+1)  
general_results_write(name=f"compas/compas", 
                      dataset_name = 'compas',
                      classifier = 'knn',
                      number_iterations=number_experiments,
                      multiplier=multiplier,
                      interior_classifier = interior_classifier, 
                      results = results)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
CPU times: user 2d 15h 15min 22s, sys: 53min 34s, total: 2d 16h 8min 57s
Wall time: 3h 55min 2s


In [8]:
unfair_discriminated = []
fair_discriminated = []
for i in range(100):
    x = np.array(benefit(results[i])['unfair_discriminated_losses']).sum()
    y = np.array(benefit(results[i])['fair_discriminated_losses']).sum()
    unfair_discriminated.append(x)
    fair_discriminated.append(y)
unfair_discriminated = np.array(unfair_discriminated)
fair_discriminated = np.array(fair_discriminated)

In [9]:
unfair_discriminated.mean(), fair_discriminated.mean()

(0.11606902915915822, 0.10412615701509409)